# **자연어 5일 - Tf-IDF Ranker**
- Document Representation : **[Source Code](http://bitly.kr/uC66hz)**
- **[스탠포드 공대 IR 수업자료](https://nlp.stanford.edu/IR-book/newslides.html)** 
- **[버지니아 공대 NLP 수업자료](http://www.cs.virginia.edu/~hw5x/Course/IR2015/_site/lectures/)**

# **PreView**

In [1]:
# 정규식에서 숫자는 : String Range 값을 특정 합니다
import os, re
fileFolder = "./News/" 
fileList   = [fileFolder + _  for _ in os.listdir(fileFolder) 
               if re.match("\d{10}.txt", _) ]

# 네이버 뉴스 수집자료를 collection 로 묶는다
collection = []
for _ in fileList:
    with open(_) as f:
        collection.append(f.read())

# DTM 자료형 생성하기
from collections import defaultdict 
DTM = defaultdict(lambda:defaultdict(int))

from konlpy.tag import Mecab
for i, doc in enumerate(collection):
    for term in Mecab().nouns(doc):
        DTM[i][term] += 1

# TDM 만들기 (DTM 의 활용)
TDM = defaultdict(lambda:defaultdict(int))
for doc, termDict in DTM.items():
    for term, freq in termDict.items():
        TDM[term][doc] = freq

list(TDM.keys())[:12]

['오류', '우회', '함수', '추가', '교수', '점', '만점', '학생', '혁진', '코리아', '텍', '설문']

In [2]:
# DTM 을 활용한 쿼리검색
query = "김정은 판문점"

searchResult = []
from nltk.tokenize import word_tokenize
for _ in word_tokenize(query):
    docResult = [] # collection 문서별 Token 검색결과
    for doc, termDict in DTM.items():
        if _ in termDict.keys():
            docResult.append(doc)
    searchResult.append(docResult)
    
# [ 김정은 검색결과,  판문점 검색결과 ]
searchResult

[[1, 36, 72, 88, 102], [1, 10, 20, 36, 50]]

In [3]:
# TDM 을 활용한 쿼리검색
query = "김정은 판문점"

searchResult = []
for _ in word_tokenize(query):
    searchResult.append(list(TDM[_].keys()))
searchResult

[[1, 36, 72, 88, 102], [1, 10, 20, 36, 50]]

In [4]:
# Vocabluary or Lexicon = in-memory 에 저장(단어 => 어느문서? 위치(포인터))
# Posting = on-disk : Huge Data (어느문서?, )
Vocabulary       = [] # TDM 저장된 단어목록
globalVocabulary = {} # in-Memory : 단어와 위치값
globalPosting    = [] # on-Dist : 디스크 문서저장

for i, doc in enumerate(collection):
    localTDM = defaultdict(int) # 문서별 TDM 계산객체
    
    # 문서별 단어의 빈도를 기록한다.
    for term in Mecab().nouns(doc):
        localTDM[term] += 1
        
    for term, freq in localTDM.items():
        if term not in Vocabulary:
            Vocabulary.append(term)
            
        termIdx = Vocabulary.index(term)
        
        if termIdx not in globalVocabulary.keys():
            nextPtr = -1 # 더이상 해당 Token 이 발견되지 않을 때
        else:
            nextPtr = globalVocabulary[termIdx] # 
        
        _posting = [i, freq, nextPtr]        
        globalVocabulary[termIdx] = len(globalPosting)
        globalPosting.append(_posting)

len(globalVocabulary), max(globalVocabulary.keys())

(5431, 5430)

In [5]:
# Inversed Index 를 활용한 Query 검색 Tree 
query = "판문점"
ptr   = globalVocabulary[Vocabulary.index(query)]
while True:
    if ptr < 0:
        break
    print(globalPosting[ptr])
    ptr = globalPosting[ptr][-1]

[50, 1, 6830]
[36, 3, 3738]
[20, 1, 2097]
[10, 1, 139]
[1, 2, -1]


In [6]:
# TDM 모델을 활용한 Query 검색엔진
# 검색결과를 빠르게 출력 합니다.
query = "김정은 판문점"

searchResult = list()
for q in word_tokenize(query):
    ptr = globalVocabulary[Vocabulary.index(q)]
    _qResult = list()
    while True:
        _posting = globalPosting[ptr]
        _qResult.append(_posting[0])
        if _posting[-1] == -1:
            break
        ptr = _posting[-1]
    searchResult.append(_qResult)

searchResult

[[102, 88, 72, 36, 1], [50, 36, 20, 10, 1]]

# **Tf IDF 를 활용한 Ranker 성능 높이기** 
- **AND NOT** 조건의 결과를 수행합니다
- **Notion of Relevance** 관계를 활용 합니다
- **[수업관련 Slice PDF](http://www.cs.virginia.edu/%7Ehw5x/Course/IR2015/_site/docs/PDFs/Boolean&VS%20model.pdf)**

## **1 tf-idf Weight Measure**
TWM : Tfidf Weight Measure : 가중치를 활용하여 Tf-idf 정규화를 진행 합니다

In [7]:
%time
# Voca or Lexicon = in-memory(단어=>어느 문서? 위치(포인터))
# Posting = on-disk:huge(어느 문서, 몇 번?)
Vocabulary = list() # 단어 목록
globalVocabulary = dict() # 메모리, 단어-위치
globalPosting = list() # 디스크
maxFreq = list() # 문서별 최대빈도

for i, doc in enumerate(collection):
    localTDM = defaultdict(int)
    for term in Mecab().nouns(doc):
        localTDM[term] += 1
    maxFreq.append(max(localTDM.values()))
    
    for term, freq in localTDM.items():
        if term not in Vocabulary:
            Vocabulary.append(term)
        termIdx = Vocabulary.index(term)
        
        if termIdx not in globalVocabulary.keys():
            nextPtr = -1
        else:
            nextPtr = globalVocabulary[termIdx]
            
        _posting = [i, freq, nextPtr]
        
        globalVocabulary[termIdx] = len(globalPosting)
        globalPosting.append(_posting)

CPU times: user 1 µs, sys: 1 µs, total: 2 µs
Wall time: 4.77 µs


In [8]:
ptr = globalVocabulary[Vocabulary.index("교수")]
while True:
    if ptr < 0:
        break
    print(globalPosting[ptr])
    ptr = globalPosting[ptr][-1]

[98, 1, 13998]
[82, 8, 10862]
[61, 1, 10472]
[59, 3, 7192]
[38, 5, 7028]
[37, 2, 5592]
[29, 1, 4759]
[25, 1, 4460]
[24, 3, 4321]
[23, 4, 1857]
[9, 2, 4]
[0, 12, -1]


In [9]:
searchResult = list()
for q in word_tokenize(query):
    ptr = globalVocabulary[Vocabulary.index(q)]
    _qResult = list()
    while True:
        _posting = globalPosting[ptr]
        _qResult.append(_posting[0])
        if _posting[-1] == -1:
            break
        ptr = _posting[-1]
    searchResult.append(_qResult)

searchResult

[[102, 88, 72, 36, 1], [50, 36, 20, 10, 1]]

## **2 TF IDF 만들기**
```
가중치 = TF * IDF
TF = freq_ij (문서번호 J 의 i 단어의 빈도) => Posting
----------------------------------------------
argmax freq_j (j번째 문서에서 최대빈도를 갖는 단어) => maxFreq
IDF = log(N => len(collection)  /  df => postring 에서 ptr 의 이동갯수)
```

In [10]:
from math import log2, sqrt

globalWeight    = globalPosting
globalDocLength = defaultdict(float)
N = len(collection)

for term in Vocabulary:
    termIdx = Vocabulary.index(term)
    ptr = globalVocabulary[termIdx]
    df  = 0
    while True: # 0:문서번호, 1:빈도, 2:위치
        df += 1
        _posting = globalPosting[ptr]
        if _posting[-1] == -1:
            break
        ptr = _posting[-1]
    IDF = log2(N/df)

    ptr = globalVocabulary[termIdx]
    while True: # 0:문서번호, 1:빈도, 2:위치
        _posting = globalPosting[ptr]
        TF = _posting[1]/maxFreq[_posting[0]]
        WEIGHT = TF * IDF
        _posting[1] = WEIGHT
        globalDocLength[_posting[0]] += WEIGHT**2
        if _posting[-1] == -1:
            break
        ptr = _posting[-1]

## **3 Maximum tf normalization 를 활용한 Query 가중치 변환**
- 질문 Query 를 **본문의 TF-IDF 가중치로** 변환 합니다
- 변환된 내용을 바탕으로 본문들의 내용을 검색 합니다.
- https://nlp.stanford.edu/IR-book/html/htmledition/maximum-tf-normalization-1.html

In [11]:
# 질문 Query 내용에 가중치를 적용 합니다
query    = "김정은 트럼프 트럼프 판문점 대한민국 대통령"
tokens   = word_tokenize(query)
queryTDM = defaultdict(int)
for _ in tokens:
    queryTDM[_] += 1
    
print(queryTDM)
queryMax = max(queryTDM.values())
queryMax

defaultdict(<class 'int'>, {'김정은': 1, '트럼프': 2, '판문점': 1, '대한민국': 1, '대통령': 1})


2

In [12]:
# Maximum tf normalization  
a = 0.5
queryWeight = defaultdict(float)
for t, f in queryTDM.items():
    TF = a + (1-a)*(f/queryMax)
    
    if t in Vocabulary:
        termIdx = Vocabulary.index(t)
        ptr = globalVocabulary[termIdx]
        df = 0
        while True:# 0:문서번호, 1:빈도, 2:위치
            df += 1
            _posting = globalPosting[ptr]
            if _posting[-1] == -1:
                break
            ptr = _posting[-1]
        IDF = log2(N/df)
        
        queryWeight[termIdx] = TF * IDF

queryWeight

defaultdict(float,
            {133: 3.273429324221892,
             131: 3.6865005271832185,
             135: 3.273429324221892,
             2164: 2.9093592038442107,
             132: 1.773429324221892})

In [13]:
# 
searchResult = defaultdict(float)
for termIdx, w in queryWeight.items():
    ptr = globalVocabulary[termIdx]
    while True:
        _posting = globalWeight[ptr]
        searchResult[_posting[0]] += w * _posting[1]
        if _posting[-1] == -1:
            break
        ptr = _posting[-1]
searchResult

defaultdict(float,
            {102: 2.041017055366818,
             88: 0.6211791038072924,
             72: 2.640074496892554,
             36: 11.662111860061826,
             1: 7.7125794934897085,
             98: 4.057051564832064,
             66: 1.270263444828736,
             50: 2.672567301264168,
             23: 4.939913396556195,
             10: 17.56515267510014,
             3: 0.6471564827105785,
             20: 0.6494145176167146,
             93: 3.761942656441342,
             83: 0.47024283205516776,
             75: 1.0259843608476387,
             68: 4.837629599116536,
             63: 0.38916648170082846,
             29: 0.49068817257930547,
             21: 0.40306528461871516,
             82: 1.0483505226700387,
             77: 4.193402090680155,
             65: 1.397800696893385,
             61: 0.4659335656311283,
             60: 0.19060918594000706,
             53: 1.630767479708949,
             46: 2.79560139378677,
             27: 0.4659335656

In [14]:
for docIdx, innerProduct in searchResult.items():
    cosine = innerProduct / sqrt(globalDocLength[docIdx])
    searchResult[docIdx] = cosine

# 여러 문서 중 유사도가 측정 가능한 문서를 출력
sorted(searchResult.items(),
       key=lambda x:x[1], reverse=True)[:10]

[(10, 1.7442066672042902),
 (36, 1.3463475558358595),
 (1, 1.3012906467221494),
 (23, 0.9910971611357137),
 (68, 0.5127828369217432),
 (98, 0.37401743610938953),
 (46, 0.3314082243996938),
 (77, 0.3277533399319457),
 (9, 0.30679076970660785),
 (50, 0.3055149067722887)]

# **유클리드를 활용한 문서 유사도 측정**
## **1 문서의 유사도 측정**

In [15]:
searchResult = defaultdict(float)

for termIdx, w in globalVocabulary.items():
    x1 = queryWeight[termIdx]
    ptr = globalVocabulary[termIdx]
    while True:
        _posting = globalWeight[ptr]
        x2 = _posting[1]
        searchResult[_posting[0]] += (x1-x2)**2
        if _posting[-1] == -1:
            break
        ptr = _posting[-1]

In [16]:
for _ in sorted(searchResult.items(), key=lambda x:x[1]):
    print(_[0], _[1], len(word_tokenize(collection[_[0]])))

74 23.16127843575197 532
92 25.08825678218501 271
78 25.578752499231122 346
11 27.8320122977396 489
62 30.699300074065857 343
67 30.787153255780606 790
23 31.698634856611957 445
9 33.03839312512632 491
83 35.42540452205284 496
60 36.4422161208503 672
27 36.66022886089046 582
80 37.06793333606439 296
21 38.239963708726584 685
66 39.47964657597552 221
69 39.51761398514184 261
63 39.88226758441258 774
71 40.78502347796054 366
38 43.35218783200697 608
58 45.991269988408696 618
3 50.09325114238466 543
22 54.83530278935449 443
48 55.234858979186455 361
20 55.5089593763493 670
85 56.23029723514068 508
1 57.86863667014972 1719
24 59.54133146601639 393
6 59.69971353622258 234
2 61.038445727461074 547
102 62.06481096418527 163
49 64.03811769339039 398
88 66.33594952021336 576
64 66.38455681770154 366
82 67.2383617652981 608
29 67.66764339366878 805
26 67.70880741913804 304
46 68.7118815105101 362
61 69.71111640616816 228
94 69.79552632700982 333
47 72.18911488820645 303
19 74.04579640385506 438


## **4 tf-idf 를 활용한 유클리드를 활용한 문서 유사도 측정 개선하기**

In [17]:
query = "일본의 수출규제 문제를 놓고 일본의 치열한 논리싸움"
query = "트럼프 문재인 판문점 판문점"
tokens = word_tokenize(query)
queryTDM = defaultdict(int)
for _ in tokens:
    queryTDM[_] += 1
queryMax = max(queryTDM.values())

a = 0.5
queryWeight = defaultdict(float)
for t, f in queryTDM.items():
    TF = a + (1-a)*(f/queryMax)
    
    if t in Vocabulary:
        queryWeight[t] = TF * 1

In [18]:
TWM = defaultdict(lambda:defaultdict(float))

for t, docDict in TDM.items():
    for d, f in docDict.items():
        TF = f/max(DTM[d].values())
        IDF = log2(N/len(docDict))
        TWM[t][d] = TF*IDF

In [19]:
searchResult = defaultdict(float)
for t, w1 in queryWeight.items():
    for d, w2 in TWM[t].items():
        searchResult[d] += w1*w2
for d, innerProduct in searchResult.items():
    docLength = sqrt(sum([TWM[t][d]**2
                          for t in list(DTM[d].keys())]))
    searchResult[d] = innerProduct / docLength
    
sorted(searchResult.items(), key=lambda x:x[1], reverse=True)

[(10, 0.2844781272830869),
 (36, 0.25349086615432354),
 (23, 0.15408840507051028),
 (1, 0.12626411294470039),
 (53, 0.09371699311651595),
 (46, 0.08849662123448318),
 (98, 0.06804853686617969),
 (50, 0.06791698589412037),
 (6, 0.06511373117295473),
 (82, 0.04129095642702978),
 (66, 0.03927511447621787),
 (16, 0.03413721084889412),
 (61, 0.030288109594095235),
 (9, 0.029790171479342677),
 (20, 0.029221609519762597),
 (68, 0.02637656696510156),
 (3, 0.021415390931196366),
 (65, 0.021227614257779606),
 (27, 0.02119880711579494),
 (40, 0.02044320593860406),
 (97, 0.00891348696399001)]

In [20]:
# Zif's Law 에서 중요도 측정
# "// flash 오류를 우회하기 위한 함수 추가" 의 '오류','우회','함수' 가중치는 0 변환
TWM["판문점"], TWM["우회"]

(defaultdict(float,
             {1: 0.15587758686770914,
              10: 0.272785777018491,
              20: 0.198389656013448,
              36: 1.1903379360806878,
              50: 0.36371436935798795}),
 defaultdict(float,
             {0: 0.0,
              1: 0.0,
              2: 0.0,
              3: 0.0,
              4: 0.0,
              5: 0.0,
              6: 0.0,
              7: 0.0,
              8: 0.0,
              9: 0.0,
              10: 0.0,
              11: 0.0,
              12: 0.0,
              13: 0.0,
              14: 0.0,
              15: 0.0,
              16: 0.0,
              17: 0.0,
              18: 0.0,
              19: 0.0,
              20: 0.0,
              21: 0.0,
              22: 0.0,
              23: 0.0,
              24: 0.0,
              25: 0.0,
              26: 0.0,
              27: 0.0,
              28: 0.0,
              29: 0.0,
              30: 0.0,
              31: 0.0,
              32: 0.0,
              33: 0.

In [21]:
collection[0]

'// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}\n\n\t\n\t교수는 스스로 7점 만점에 6.3점…학생은 4.3점 줘고혁진 코리아텍 교수 설문조사 결과…연구환경 인식 차이 커\'불 꺼진 교수실\'[연합뉴스 자료 사진](대전=연합뉴스) 이재림 기자 = 국내 청년 과학자와 교수 간 연구 환경에 대한 인식차가 적지 않은 것으로 나타났다. 진로에 대한 관심 여부는 그 간극이 크게 벌어져 있는 것으로 파악됐다.    25일 한국연구재단(NRF)에 따르면 고혁진 한국산업기술대학교(코리아텍) 교수를 비롯한 재단 정책혁신팀은 연구환경에 대한 이해당사자 간 인식 차이 조사 심층진단 내용을 \'NRF 이슈리포트\'로 발표했다.    4월 17∼25일 이공계 대학원생 및 박사 후 연구원(청년과학자) 3천301명을 상대로, 5월 2∼16일 이공분야 지원사업 수행 교수(연구책임자) 2천488명을 상대로 각각 설문 조사했다.    연구실 문화를 살필 수 있는 문항에 대한 응답 분석 결과 모든 항목에서 교수가 학생보다 긍정적이었다.    교수의 경우는 긍정적 응답 비율이 부정적인 것보다 압도적으로 높았다.    학생 인식 수준을 기준으로 한 인식 수준 표준화 값(교수 인식 수준-학생 인식 수준/학생 인식 수준)을 보면 두 집단의 인식 차이는 24.8% 정도다.    특히 진로에 대한 적극적인 관심에 대한 차이는 46.5%나 됐다.     \'지도교수는 제자의 진로에 대해 적극적인 관심을 갖고 있다\'라는 문항에 교수는 6.3점을, 학생은 4.3점(이상 7점 만점)을 각각 매겼다.    이는 지난해 국제 학술지 네이처(nature)가 \'연구실 리더십 문제\'(Leadership problem In The Lab)에서 공개한 외국 사례를 크게 웃도는 수치다. 외국에서 연구책임자와 연구원 간 진로 상담 인식 차이는 28%였다.    국내의 경우 동료·선배와의 차별 여부(32.8%)와 과제 참여를 통한 경제적 보상(30.4